In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd

import os
import warnings
warnings.filterwarnings('ignore')

import json

import math

import tensorflow as tf
from tensorflow.keras import layers as k
from keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import matplotlib.pyplot as plt
import seaborn as sns

from keras.preprocessing.image import ImageDataGenerator


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
image_size = (299, 299, 3)
batch_num = 128
epoch_num = 30

# create_model_name
model = 'InceptionV3_generator'
numbering = '02_1'
preprocessing_method ='only_gen'
class_type = 'all_class'
runtime_type = 'tpu'

# 고정 내역
max_pic_cnt = 1000
top_layer = 'customized_top_layer'

if runtime_type == 'tpu':
    model_name = f'{model}_{numbering}_{preprocessing_method}_{class_type}_batch({batch_num})_epoch({epoch_num})_tpu'
else :
    model_name = f'{model}_{numbering}_{preprocessing_method}_{class_type}_batch({batch_num})_epoch({epoch_num})'
print(model_name)

InceptionV3_generator_02_1_only_gen_all_class_batch(128)_epoch(30)_tpu


In [ ]:
# create model folder
base_path = f'/content/drive/MyDrive/project3/image_model/{model}'
if not os.path.exists(base_path):
    os.makedirs(base_path)

model_path = f'/content/drive/MyDrive/project3/image_model/{model}/{model_name}'
if not os.path.exists(model_path):
    os.makedirs(model_path)

# result folder
result_path = f'/content/drive/MyDrive/project3/image_model/{model}/{model_name}/result'
if not os.path.exists(result_path):
    os.makedirs(result_path)

In [ ]:
df = pd.read_pickle('/content/drive/MyDrive/project3/data/traindata/read_image.pkl')
df.head()

,file_path,type,middle_class,small_class,food_class,group_number,crop_area
0,/content/drive/MyDrive/project3/data/traindata...,raw,구이,갈비구이,구이/갈비구이,1,NaN
1,/content/drive/MyDrive/project3/data/traindata...,raw,구이,갈비구이,구이/갈비구이,2,NaN
2,/content/drive/MyDrive/project3/data/traindata...,raw,구이,갈비구이,구이/갈비구이,3,NaN
3,/content/drive/MyDrive/project3/data/traindata...,raw,구이,갈비구이,구이/갈비구이,4,NaN
4,/content/drive/MyDrive/project3/data/traindata...,raw,구이,갈비구이,구이/갈비구이,5,NaN


In [ ]:
df.shape

(150283, 7)

In [ ]:
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['small_class'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['small_class'])

In [ ]:
train_df.shape, val_df.shape, test_df.shape

((120226, 7), (15028, 7), (15029, 7))

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import cv2

def process_image(image_array, new_ratio=1.5, new_image_size=299):
    height, width, _ = image_array.shape

    if width == height: # 이미지 넓이와 높이가 같다면 모델 인풋 사이즈로 사이즈만 바꿔줌
        new_image = cv2.resize(image_array, (new_image_size, new_image_size), interpolation=cv2.INTER_LINEAR)
        return new_image

    ratio = width / height if width > height else height / width

    if ratio > new_ratio: # 가로, 세로가 1.5배 차이 나면 사용 x
        return None

    if width > height: # 짧은 변의 길이에 맞춰서 가운데서 양쪽을 자름
        new_width = int(height)
        left = (width - new_width) / 2
        right = (width + new_width) / 2
        top = 0
        bottom = height
    else:
        new_height = int(width)
        left = 0
        right = width
        top = (height - new_height) / 2
        bottom = (height + new_height) / 2

    cropped_image = image_array[int(top):int(bottom), int(left):int(right)]

    new_size = max(cropped_image.shape[:2])

    black_background = np.full((new_size, new_size, 3), 0, dtype=np.uint8) # 미세하게 다를 수 있기 때문에 뒤에 검은색 배경으로 패딩해줌
    black_y_offset = (new_size - cropped_image.shape[0]) // 2
    black_x_offset = (new_size - cropped_image.shape[1]) // 2

    black_background[black_y_offset:black_y_offset+cropped_image.shape[0],
                     black_x_offset:black_x_offset+cropped_image.shape[1]] = cropped_image

    new_image = cv2.resize(black_background, (new_image_size, new_image_size), interpolation=cv2.INTER_LINEAR)
    new_image = new_image.astype(np.uint8)

    return new_image

class CustomImageDataGenerator(ImageDataGenerator):
    def flow_from_directory(self, directory, target_size=(299, 299), *args, **kwargs):
        generator = super().flow_from_directory(directory, target_size=target_size, *args, **kwargs)
        while True:
            batch_x, batch_y = next(generator)
            new_batch_x = []
            new_batch_y = []
            for i in range(len(batch_x)):
                img = load_img(generator.filepaths[generator.index_array[i]])
                img_array = img_to_array(img)
                processed_img = process_image(img_array, new_image_size=target_size[0])
                if processed_img is not None:
                    new_batch_x.append(processed_img)
                    new_batch_y.append(batch_y[i])
            if new_batch_x:
                yield np.array(new_batch_x), np.array(new_batch_y)

 # 데이터 생성기 설정


In [ ]:
# Generator1
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20, #회전 각도
    width_shift_range=0.2, #이미지를 임의의 가로 방향으로 이동
    height_shift_range=0.2, #이미지를 임의의 세로 방향으로 이동
    shear_range=0.2, #반시계방향으로 밀어 적용
    zoom_range=0.2, #확대 비율
    horizontal_flip=True,
    fill_mode='nearest'
)

# Generator2
# train_datagen = ImageDataGenerator(rescale=1./255)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
new_image_size = (299, 299)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='file_path',
    y_col='small_class',
    target_size=new_image_size,
    batch_size=batch_num,
    class_mode='categorical',
    shuffle=True
)

val_generator = val_datagen.flow_from_dataframe(
    val_df,
    x_col='file_path',
    y_col='small_class',
    target_size=new_image_size,
    batch_size=batch_num,
    class_mode='categorical',
    shuffle=True
)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='file_path',
    y_col='small_class',
    target_size=new_image_size,
    batch_size=batch_num,
    class_mode='categorical',
    shuffle=False
)

Found 120088 validated image filenames belonging to 150 classes.
Found 15016 validated image filenames belonging to 150 classes.
Found 15010 validated image filenames belonging to 150 classes.


In [ ]:
num_classes = len(train_generator.class_indices)
print("Number of classes:", num_classes)

Number of classes: 150


In [ ]:
# Load the InceptionV3
# model pre-trained -> by imagenet dataset
# without the top layer -> 마지막 결과를 내는 레이어 층에 관한
inception = InceptionV3(weights='imagenet', input_shape=[299, 299, 3], include_top=False)
inception.trainable = False  # Freeze the layers of InceptionV3

# Define the Customized top layer
model = tf.keras.models.Sequential([
    inception,
    k.GlobalAveragePooling2D(),
    k.Dropout(0.2),
    k.Dense(512, activation='relu'),
    k.BatchNormalization(),
    k.Dropout(0.1),
    k.Dense(256, activation='relu'),
    k.BatchNormalization(),
    k.Dropout(0.1),
    k.Dense(num_classes, activation='softmax')
])

print(model.summary())



87910968/87910968 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 8, 8, 2048)        21802784  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 batch_normalization_94 (Ba  (None, 512)               2048      
 tchNormalization)                                               
                                                         

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint(filepath=os.path.join(model_path, f'{model_name}_best.h5'),
                                   monitor='val_loss', save_best_only=True)



model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=[tf.keras.metrics.CategoricalAccuracy(), tf.keras.metrics.Precision(name='precision'), tf.keras.metrics.Recall(name='recall')]
)

history = model.fit_generator(train_generator,
                              epochs=epoch_num,
                              steps_per_epoch= len(train_generator),
                              validation_data = val_generator,
                              validation_steps= len(val_generator),
                              use_multiprocessing=True,
                              callbacks=[model_checkpoint, early_stopping]
                              )

Epoch 1/30
939/939 [==============================] - 34189s 36s/step - loss: 2.2110 - categorical_accuracy: 0.4348 - precision: 0.7462 - recall: 0.2455 - val_loss: 1.6151 - val_categorical_accuracy: 0.5583 - val_precision: 0.7734 - val_recall: 0.4071
Epoch 2/30
939/939 [==============================] - 3974s 4s/step - loss: 1.7484 - categorical_accuracy: 0.5277 - precision: 0.7555 - recall: 0.3682 - val_loss: 1.4933 - val_categorical_accuracy: 0.5869 - val_precision: 0.7867 - val_recall: 0.4447
Epoch 3/30
939/939 [==============================] - 3990s 4s/step - loss: 1.6563 - categorical_accuracy: 0.5465 - precision: 0.7625 - recall: 0.3967 - val_loss: 1.4520 - val_categorical_accuracy: 0.5928 - val_precision: 0.7800 - val_recall: 0.4628
Epoch 4/30
939/939 [==============================] - 3998s 4s/step - loss: 1.6087 - categorical_accuracy: 0.5582 - precision: 0.7668 - recall: 0.4103 - val_loss: 1.4108 - val_categorical_accuracy: 0.6044 - val_precision: 0.7949 - val_recall: 0.463

In [ ]:
# model save
model.save(os.path.join(model_path, f'{model_name}.h5'))


# training history save
with open(os.path.join(result_path, f'{model_name}_history.json'), 'w') as f:
    json.dump(history.history, f)

In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['categorical_accuracy'], label='Train Accuracy')
plt.plot(history.history['val_categorical_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Accuracy By Epochs')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss By Epochs')

plt.suptitle(f'{model_name}')

# save training_plot
plt.savefig(os.path.join(result_path, f'{model_name}_training_plot.png'))
plt.show()

In [ ]:
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = test_generator.classes

conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)

# Print classification report for detailed metrics
class_report = classification_report(y_true_classes, y_pred_classes, target_names=test_generator.class_indices.keys())
print(class_report)

with open(os.path.join(result_path, f'{model_name}_classification_report.txt'), 'w') as f:
    f.write(class_report)

# Plot the confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=test_generator.class_indices.keys(), yticklabels=test_generator.class_indices.keys())
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title(f'{model_name} Confusion Matrix')

plt.savefig(os.path.join(result_path, f'{model_name}_confusion_matrix.png'))
plt.show()

In [ ]:

# Calculate accuracy using sklearn
accuracy = accuracy_score(y_true_classes, y_pred_classes)
print('Accuracy:', accuracy)

# Calculate precision and recall using sklearn
precision = precision_score(y_true_classes, y_pred_classes, average='weighted')
recall = recall_score(y_true_classes, y_pred_classes, average='weighted')
print('Precision:', precision)
print('Recall:', recall)

# Append accuracy to a text file
with open(os.path.join(base_path, 'accuracy.txt'), 'a') as f:
    f.write(f'{model_name}_Accuracy: {accuracy:.2f}%\n')
    f.write(f'{model_name}_Precision: {precision:.2f}%\n')
    f.write(f'{model_name}_Recall: {recall:.2f}%\n')

In [ ]:
val_prediction = model.predict_generator(val_generator, steps = len(val_generator))

In [ ]:
food_classes = val_generator.class_indices
print(food_classes)

In [ ]:
# y가 예측한 레이블
val_y_pred = np.argmax(val_prediction, axis = 1)
# 실제 y 값
val_y_true = val_generator.classes

print("val_y_pred ===  ", val_y_pred[:50])
print("val_y_true ===  ", val_y_true[:50])

In [ ]:
# confusion_matrix
food_confusion_matrix = confusion_matrix(val_y_true, val_y_pred)
print(food_confusion_matrix)
print("shape==", food_confusion_matrix.shape)

In [ ]:
import seaborn as sns
import pandas as pd

# 단순히 갯수로 비교하면 이미지가 많은 클래스가 상대적으로 나쁘게 보일 수 있다.
# 현재 validation set의 class별 이미지 갯수는 전처리 과정에서 일부 누락된 게 있어 모두 100개가 아님
# --> 비율로 confusion_matrix를 normalize한다!

row_sums = food_confusion_matrix.sum(axis=1, keepdims=True)
norm_conf_mx = food_confusion_matrix / row_sums

df_cm = pd.DataFrame(norm_conf_mx, index = range(150),columns=range(150))

plt.figure(figsize = (20,14))
plt.title("confusion_matrix (x : Predicted, y : Actual) ")

sns.heatmap(df_cm, annot=False, cmap="Blues")

In [ ]:
# 각 index별 음식 이름
food_classes = val_generator.class_indices
print(food_classes)

In [ ]:
# food_dict -> key : 음식 이름, value : index인 dictionary
food_dict = val_generator.class_indices
# foods_prob : (음식 이름, 음식의 index, 예측값=실제값일 확률)인 tuple을 각 원소로 갖는 list
foods_prob = [(food_name,idx, prob) for food_name, idx, prob in zip(food_dict.keys(), food_dict.values() ,np.diag(norm_conf_mx))]
foods_prob =  sorted(foods_prob,key= lambda x : x[2])
print(foods_prob)

In [ ]:
# 잘 분류될 확률이 60% 이하인 음식들
bad_classfifed_foods = [food_prob for food_prob in foods_prob if food_prob[2] <= 0.6 ]
bad_classfifed_foods